In [1]:
import pandas as pd
import altair as alt
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [2]:
# # Uncomment this to reload the data from the original source. This can take a few seconds.
# dfs = pd.read_excel("su-d-01.06.01.01.10.xlsx", sheet_name=[str(year) for year in range(1987, 2021)], skiprows=2, skipfooter=5, index_col=0)
# marriages = []

# for year, df in dfs.items():
#     _df = df.drop(labels=["Total"], axis=0).drop(labels=["Total"], axis=1)
#     for idx, val in _df.loc["Schweiz"].items():
#         marriages.append((idx, val, year, 'm'))
#     for idx, val in _df["Schweiz"].items():
#         marriages.append((idx, val, year, 'f'))

# df = pd.DataFrame(marriages, columns="Nationality Marriages Year Sex_ch".split())
# df.to_csv("marriages_ch.csv")

In [3]:
df = pd.read_csv("marriages_ch.csv", index_col=0)
df['Marriages'] = pd.to_numeric(df['Marriages'], errors='coerce')
# Some countries have a footnote in the source. Thats why they are imported as Serbia 1, Serbia 2 etc.
df['Nationality'] = df['Nationality']\
   .apply(lambda x: x.replace("1", ""))\
   .apply(lambda x: x.replace("2", ""))\
   .apply(lambda x: x.replace("3", ""))\
   .apply(lambda x: x.strip())
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13830 entries, 0 to 13901
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Nationality  13830 non-null  object 
 1   Marriages    13830 non-null  float64
 2   Year         13830 non-null  int64  
 3   Sex_ch       13830 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 540.2+ KB


In [4]:
df_filtered = df[df['Nationality'] != 'Schweiz']

top_5_f_2020 = df_filtered[(df_filtered['Year'] == 2020) & (df_filtered['Sex_ch'] == 'f')] \
    .nlargest(5, 'Marriages')['Nationality'].tolist()
top_5_f_2020 += ["Serbien und Montenegro"]

top_5_m_2020 = df_filtered[(df_filtered['Year'] == 2020) & (df_filtered['Sex_ch'] == 'm')] \
    .nlargest(5, 'Marriages')['Nationality'].tolist()
top_5_m_2020 += ["Brasilien"]

countries = list(sorted(set(top_5_f_2020 + top_5_m_2020)))
colors = ['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00','#a65628','#f781bf','#999999']
country_colors = {country: color for country, color in zip(countries, colors)}

colors_men = [country_colors[country] for country in top_5_m_2020]
colors_women = [country_colors[country] for country in top_5_f_2020]

In [5]:
alt.Chart(df_filtered).mark_line(point=True).encode(
    x=alt.X("Year:O", title="Jaar"),
    y=alt.Y("rank:O", title="Rang"),
    color=alt.Color('Nationality:N', legend=alt.Legend(title="Nationaliteit")).scale(domain=top_5_f_2020, range=colors_women)
).transform_filter(
    alt.datum.Sex_ch == 'f'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("Marriages", order="descending")],
    groupby=["Year"]
).transform_filter(
    alt.FieldOneOfPredicate(field='Nationality', oneOf=top_5_f_2020)
).transform_filter(
    "datum.Nationality != 'Serbien und Montenegro' || (datum.Year >= 1994 && datum.Year <= 2009)"
).properties(
    title="Top 5 huwelijken van een Zwitserse vrouw met een buitenlandse man",
    width=600,
    height=150
)

alt.Chart(...)

In [6]:
alt.Chart(df_filtered).mark_line(point=True).encode(
    x=alt.X("Year:O", title="Jaar"),
    y=alt.Y("rank:O", title="Rang").scale(domain=list(range(1, 11))),
    color=alt.Color('Nationality:N', legend=alt.Legend(title="Nationaliteit")).scale(domain=top_5_m_2020, range=colors_men)
).transform_filter(
    alt.datum.Sex_ch == 'm'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("Marriages", order="descending")],
    groupby=["Year"]
).transform_filter(
    {'field': 'Nationality', 'oneOf': top_5_m_2020}
).properties(
    title="Top 6 huwelijken van een Zwitserse man met een buitenlandse vrouw",
    width=600,
    height=150,
)

alt.Chart(...)

In [7]:
# List of countries including Thailand
countries_with_thailand = ['Thailand'] + [c for c in countries if c != 'Thailand' and c != "Serbien und Montenegro"]
colors_with_thailand = [country_colors[country] for country in countries_with_thailand if country != "Serbien und Montenegro"]
df_filtered.loc["Sex_ch"] = df_filtered["Sex_ch"].apply(lambda x: 'v' if x == 'f' else x)

# Left chart: Including Thailand
chart_with_thailand = alt.Chart(df_filtered).mark_line(point=True).encode(
    x=alt.X("Sex_ch:O", title="Geslacht Zwitser", axis=alt.Axis(labelAngle=0)),
    y=alt.Y("rank:Q", title="Rang").scale(reverse=True),
    color=alt.Color('Nationality:N', legend=alt.Legend(title="Nationaliteit")).scale(domain=countries_with_thailand, range=colors_with_thailand)
).transform_window(
    rank="rank()",
    sort=[alt.SortField("Marriages", order="descending")],
    groupby=["Year", "Sex_ch"]
).transform_filter(
    {'field': 'Nationality', 'oneOf': countries_with_thailand}
).transform_filter(
    (alt.datum.Year == 2020) & (alt.datum.Nationality != "Serbien und Montenegro")
).properties(
    title="Inclusief Thailand",
)

# Right chart: Excluding Thailand
countries_without_thailand = [c for c in countries if c != 'Thailand' and c != "Serbien und Montenegro"]
colors_without_thailand = [country_colors[country] for country in countries_without_thailand]
chart_without_thailand = alt.Chart(df_filtered).mark_line(point=True).encode(
    x=alt.X("Sex_ch:N", title="Geslacht Zwitser", axis=alt.Axis(labelAngle=0)),
    y=alt.Y("rank:Q", title="Rang").scale(reverse=True),
    color=alt.Color(
        'Nationality:N', 
        legend=alt.Legend(title="Nationaliteit"))\
            .scale(domain=countries_without_thailand, range=colors_without_thailand
    )
).transform_window(
    rank="rank()",
    sort=[alt.SortField("Marriages", order="descending")],
    groupby=["Year", "Sex_ch"]
).transform_filter(
    {'field': 'Nationality', 'oneOf': countries_without_thailand}
).transform_filter(
    (alt.datum.Year == 2020) & (alt.datum.Nationality != "Serbien und Montenegro")
).properties(
    title="Exclusief Thailand",
)

# Combine the two charts side by side
(chart_with_thailand | chart_without_thailand).properties(
    title='Populariteit van buitenlandse partners in Zwitserse huwelijken in 2020'
)


/tmp/ipykernel_568192/3979299474.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc["Sex_ch"] = df_filtered["Sex_ch"].apply(lambda x: 'v' if x == 'f' else x)


alt.HConcatChart(...)